In [1]:
# Importation des bibliothèques nécessaires pour le traitement du langage naturel et l'apprentissage automatique
import nltk
import os
import csv
import math
import random
import logging
import itertools
import time
import re
import torch
import numpy as np
from tqdm import tqdm
from dataclasses import dataclass
from typing import Optional, Tuple
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import (
    BertModel, BertConfig, BertPreTrainedModel,
    BertTokenizer, AdamW, get_linear_schedule_with_warmup
)
from nltk.corpus import wordnet as wn
from torch.nn.functional import softmax
from tabulate import tabulate

# Configuration de la journalisation pour un suivi détaillé des opérations
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

#Téléchargement des ressources WordNet nécessaires
nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')
nltk.data.path.append('/kaggle/working/nltk_data')

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...


In [2]:
#Google Colab Unzipping Methods
#Kaggle Notebook Unzipping Methods
#Python's zipfile module

import zipfile

with zipfile.ZipFile('/kaggle/working/nltk_data/corpora/wordnet.zip', 'r') as zip_ref:
    zip_ref.extractall('//kaggle/working/nltk_data/corpora/')

In [3]:
# Structure représentant un enregistrement pour la sélection de gloses
from collections import namedtuple
GlossSelectionRecord = namedtuple("GlossSelectionRecord", ["identifiant", "phrase", "cles_sens", "glosses", "cibles"])
EntreesBert = namedtuple("EntreesBert", ["ids_entree", "masque_entree", "ids_segment", "id_etiquette"])

# Définition des parties du discours pour WordNet
PARTIES_DISCOURS_WORDNET = {'VERBE': wn.VERB, 'NOM': wn.NOUN, 'ADJECTIF': wn.ADJ, 'ADVERBE': wn.ADV}

def obtenir_glosses(lemme, pos):
    """
    Récupère les définitions pour un mot donné dans WordNet.

    L'utilisateur fournit un lemme et une partie du discours,
    la fonction renvoie un dictionnaire de définitions.
    """
    resultats = dict()
    pos_wordnet = PARTIES_DISCOURS_WORDNET.get(pos, None) if pos is not None else None
    morphemes = wn._morphy(lemme, pos=pos_wordnet) if pos is not None else []

    for synset in set(wn.synsets(lemme, pos=pos_wordnet)):
        cle_sens = None
        for lemme_synset in synset.lemmas():
            if lemme_synset.name().lower() == lemme.lower():
                cle_sens = lemme_synset.key()
                break
            elif lemme_synset.name().lower() in morphemes:
                cle_sens = lemme_synset.key()

        if cle_sens is not None:
            resultats[cle_sens] = synset.definition()

    return resultats

class JeuDonneesWSD(Dataset):
    """Jeu de données personnalisé pour la désambiguïsation lexicale"""
    def __init__(self, caracteristiques):
        self.caracteristiques = caracteristiques

    def __getitem__(self, index):
        return self.caracteristiques[index]

    def __len__(self):
        return len(self.caracteristiques)

In [4]:
class BertWSD(BertPreTrainedModel):
    """
    Modèle BERT spécialisé pour la désambiguïsation lexicale.

    Le modèle étend BertPreTrainedModel avec une couche de classification linéaire.
    """
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.couche_classement = torch.nn.Linear(config.hidden_size, 1)
        self.init_weights()

def calculer_perte_ponderee(perte, facteur_ponderation):
    """
    Calcule une perte pondérée avec un facteur de correction.

    Permet d'ajuster la contribution de la perte lors de l'entraînement.
    """
    carre_facteur = facteur_ponderation ** 2
    return 1 / (2 * carre_facteur) * perte + math.log(1 + carre_facteur)

def tronquer_sequence_paire(jetons_a, jetons_b, longueur_max):
    """
    Tronque une paire de séquences à une longueur maximale.

    Assure que la longueur totale des séquences ne dépasse pas la limite.
    """
    while True:
        longueur_totale = len(jetons_a) + len(jetons_b)
        if longueur_totale <= longueur_max:
            break
        if len(jetons_a) > len(jetons_b):
            jetons_a.pop()
        else:
            jetons_b.pop()

In [5]:
from dataclasses import dataclass
from typing import Optional, Tuple

@dataclass
class ConfigurationEchantillonnage:
    """Configuration pour l'échantillonnage des données.
    
    Attributes:
        taille_max: Nombre maximum d'échantillons à prendre
        plage: Tuple optionnel (debut, fin) pour spécifier une plage de lignes
    """
    taille_max: Optional[int] = None
    plage: Optional[Tuple[int, int]] = None

def creer_enregistrements_depuis_csv(chemin_csv, fonction_deserialisation, config_echantillonnage=None):
    """
    Crée des enregistrements à partir d'un fichier CSV avec options d'échantillonnage avancées.
    
    Args:
        chemin_csv: Chemin vers le fichier CSV
        fonction_deserialisation: Fonction pour convertir les lignes CSV
        config_echantillonnage: Configuration d'échantillonnage (taille_max et plage)
        
    Returns:
        Liste des enregistrements échantillonnés
    """
    enregistrements = []
    
    with open(chemin_csv, 'r', encoding='utf-8', newline='') as fichier:
        lecteur = csv.reader(fichier)
        next(lecteur)  # Ignorer l'en-tête
        
        # Convertir l'itérateur en liste pour permettre l'indexation
        toutes_lignes = list(lecteur)
        
        # Déterminer les indices de début et fin
        debut = 0
        fin = len(toutes_lignes)
        
        if config_echantillonnage and config_echantillonnage.plage:
            debut = max(0, config_echantillonnage.plage[0] - 1)  # -1 car les utilisateurs comptent à partir de 1
            fin = min(len(toutes_lignes), config_echantillonnage.plage[1])
        
        # Appliquer la limitation de taille si spécifiée
        if config_echantillonnage and config_echantillonnage.taille_max:
            fin = min(fin, debut + config_echantillonnage.taille_max)
        
        # Créer les enregistrements pour la plage sélectionnée
        for ligne in toutes_lignes[debut:fin]:
            enregistrements.append(fonction_deserialisation(ligne))
            
        logger.info(f"Échantillonnage effectué - Plage : {debut+1} à {fin} - "
                   f"Nombre d'enregistrements : {len(enregistrements)}")
    
    return enregistrements

def deserialiser_enregistrement_csv(ligne):
    """
    Convertit une ligne CSV en enregistrement de sélection de gloses.
    """
    return GlossSelectionRecord(
        ligne[0],  # identifiant
        ligne[1],  # phrase
        eval(ligne[2]),  # cles_sens
        eval(ligne[3]),  # glosses
        [int(t) for t in eval(ligne[4])]  # cibles
    )

def charger_jeu_donnees(
    chemin_csv,
    tokeniseur,
    longueur_sequence_max,
    debut_plage=None,
    fin_plage=None,
    taille_max=None
):
    """
    Charge un jeu de données avec options d'échantillonnage avancées.
    
    Args:
        chemin_csv: Chemin vers le fichier CSV
        tokeniseur: Tokeniseur BERT
        longueur_sequence_max: Longueur maximale des séquences
        debut_plage: Indice de début pour l'échantillonnage (commençant à 1)
        fin_plage: Indice de fin pour l'échantillonnage
        taille_max: Nombre maximum d'échantillons à prendre
    """
    config = ConfigurationEchantillonnage(
        taille_max=taille_max,
        plage=(debut_plage, fin_plage) if debut_plage and fin_plage else None
    )
    
    enregistrements = creer_enregistrements_depuis_csv(
        chemin_csv,
        deserialiser_enregistrement_csv,
        config
    )
    
    caracteristiques = creer_caracteristiques_depuis_enregistrements(
        enregistrements,
        longueur_sequence_max,
        tokeniseur
    )
    
    logger.info(f"Chargé {len(caracteristiques)} échantillons depuis {chemin_csv}")
    
    return JeuDonneesWSD(caracteristiques)

def creer_caracteristiques_depuis_enregistrements(enregistrements, longueur_seq_max, tokeniseur):
    """
    Convertit les enregistrements en caractéristiques pour BERT.

    Prépare les données d'entrée pour le modèle en tokenisant et formatant.
    """
    caracteristiques = []
    for enregistrement in tqdm(enregistrements, desc="Conversion des données"):
        jetons_a = tokeniseur.tokenize(enregistrement.phrase)
        sequences = [(gloss, 1 if i in enregistrement.cibles else 0) for i, gloss in enumerate(enregistrement.glosses)]

        paires = []
        for seq, etiquette in sequences:
            jetons_b = tokeniseur.tokenize(seq)
            tronquer_sequence_paire(jetons_a, jetons_b, longueur_seq_max - 3)

            jetons = jetons_a + ['[SEP]']
            ids_segment = [0] * len(jetons)

            jetons += jetons_b + ['[SEP]']
            ids_segment += [1] * (len(jetons_b) + 1)

            jetons = ['[CLS]'] + jetons
            ids_segment = [0] + ids_segment

            ids_entree = tokeniseur.convert_tokens_to_ids(jetons)
            masque_entree = [1] * len(ids_entree)

            longueur_padding = longueur_seq_max - len(ids_entree)
            ids_entree += [0] * longueur_padding
            masque_entree += [0] * longueur_padding
            ids_segment += [0] * longueur_padding

            assert len(ids_entree) == longueur_seq_max
            assert len(masque_entree) == longueur_seq_max
            assert len(ids_segment) == longueur_seq_max

            paires.append(
                EntreesBert(ids_entree=ids_entree, masque_entree=masque_entree,
                            ids_segment=ids_segment, id_etiquette=etiquette)
            )

        caracteristiques.append(paires)

    return caracteristiques

def regrouper_lots(lot):
    """
    Regroupe les lots de données pour l'entraînement et l'évaluation.

    Prépare les tenseurs pour l'entrée du modèle BERT.
    """
    longueur_seq_max = len(lot[0][0].ids_entree)

    lots_regroupes = []
    for sous_lot in lot:
        taille_lot = len(sous_lot)
        sous_lots_regroupes = [torch.zeros([taille_lot, longueur_seq_max], dtype=torch.long) for _ in range(3)] + \
                               [torch.zeros([taille_lot], dtype=torch.long)]

        for i, entree_bert in enumerate(sous_lot):
            sous_lots_regroupes[0][i] = torch.tensor(entree_bert.ids_entree, dtype=torch.long)
            sous_lots_regroupes[1][i] = torch.tensor(entree_bert.masque_entree, dtype=torch.long)
            sous_lots_regroupes[2][i] = torch.tensor(entree_bert.ids_segment, dtype=torch.long)
            sous_lots_regroupes[3][i] = torch.tensor(entree_bert.id_etiquette, dtype=torch.long)

        lots_regroupes.append(sous_lots_regroupes)

    return lots_regroupes

In [6]:
def passe_avant_selection_gloses(modele, lots, appareil):
    """
    Réalise une passe avant pour la sélection de gloses.

    Le système calcule la perte et les logits pour chaque lot de données
    en utilisant le modèle BERT spécialisé.
    """
    perte_lot = 0
    liste_logits = []
    fonction_perte = torch.nn.CrossEntropyLoss()

    for lot in lots:
        lot = tuple(t.to(appareil) for t in lot)
        sorties = modele.bert(input_ids=lot[0], attention_mask=lot[1], token_type_ids=lot[2])
        etat_cache = modele.dropout(sorties[1])

        logits = modele.couche_classement(etat_cache).squeeze(-1)
        etiquettes = torch.max(lot[3], -1).indices.detach()
        perte_lot += fonction_perte(logits.unsqueeze(dim=0), etiquettes.unsqueeze(dim=-1))
        liste_logits.append(logits)

    perte = perte_lot / len(lots)
    return perte, liste_logits

def entrainer_wsd(
    chemin_entrainement,
    chemin_evaluation,
    repertoire_sortie= '/kaggle/working/resultats',
    debut_plage_entrainement=None,
    fin_plage_entrainement=None,
    taille_max_entrainement=None,
    debut_plage_evaluation=None,
    fin_plage_evaluation=None,
    taille_max_evaluation=None
):
    """
    Entraîne un modèle de désambiguïsation lexicale avec options d'échantillonnage avancées.
    
    Args:
        chemin_entrainement: Chemin vers les données d'entraînement
        chemin_evaluation: Chemin vers les données d'évaluation
        repertoire_sortie: Répertoire pour sauvegarder le modèle
        debut_plage_entrainement: Première ligne à inclure pour l'entraînement
        fin_plage_entrainement: Dernière ligne à inclure pour l'entraînement
        taille_max_entrainement: Nombre maximum d'échantillons d'entraînement
        debut_plage_evaluation: Première ligne à inclure pour l'évaluation
        fin_plage_evaluation: Dernière ligne à inclure pour l'évaluation
        taille_max_evaluation: Nombre maximum d'échantillons d'évaluation
    """
    # Configuration des hyperparamètres
    longueur_sequence_max = 128
    taille_lot = 8
    nombre_epoques = 3
    taux_apprentissage = 5e-5
    graine = 42

    # Configuration de la reproductibilité
    random.seed(graine)
    np.random.seed(graine)
    torch.manual_seed(graine)
    torch.cuda.manual_seed_all(graine)

    # Sélection et configuration du périphérique
    appareil = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logger.info(f"Utilisation du périphérique : {appareil}")

    # Chargement du modèle et du tokeniseur
    nom_modele = '/kaggle/input/modell'
    configuration = BertConfig.from_pretrained(nom_modele, num_labels=2)
    tokeniseur = BertTokenizer.from_pretrained(nom_modele)
    modele = BertWSD.from_pretrained(nom_modele, config=configuration)

    # Gestion des tokens spéciaux
    if '[TGT]' not in tokeniseur.additional_special_tokens:
        tokeniseur.add_special_tokens({'additional_special_tokens': ['[TGT]']})
        modele.resize_token_embeddings(len(tokeniseur))

    modele.to(appareil)

    # Chargement des données d'entraînement avec les nouvelles options d'échantillonnage
    logger.info("Chargement des données d'entraînement...")
    jeu_donnees_entrainement = charger_jeu_donnees(
        chemin_entrainement,
        tokeniseur,
        longueur_sequence_max,
        debut_plage=debut_plage_entrainement,
        fin_plage=fin_plage_entrainement,
        taille_max=taille_max_entrainement
    )

    # Création du DataLoader pour l'entraînement
    echantillonneur = RandomSampler(jeu_donnees_entrainement)
    chargeur_donnees_entrainement = DataLoader(
        jeu_donnees_entrainement,
        sampler=echantillonneur,
        batch_size=taille_lot,
        collate_fn=regrouper_lots
    )

    # Configuration de l'optimiseur avec gestion du weight decay
    no_decay = ['bias', 'LayerNorm.weight']
    parametres_optimiseur = [
        {
            'params': [p for n, p in modele.named_parameters() if not any(nd in n for nd in no_decay)],
            'weight_decay': 0.01
        },
        {
            'params': [p for n, p in modele.named_parameters() if any(nd in n for nd in no_decay)],
            'weight_decay': 0.0
        }
    ]

    # Initialisation de l'optimiseur et du plannificateur
    nombre_etapes_totales = len(chargeur_donnees_entrainement) * nombre_epoques
    optimiseur = AdamW(parametres_optimiseur, lr=taux_apprentissage)
    plannificateur = get_linear_schedule_with_warmup(
        optimiseur,
        num_warmup_steps=0,
        num_training_steps=nombre_etapes_totales
    )

    # Début de l'entraînement
    logger.info("\n========== Début de l'entraînement ==========")
    logger.info(f"Nombre d'échantillons d'entraînement : {len(jeu_donnees_entrainement)}")
    logger.info(f"Nombre d'époques : {nombre_epoques}")
    logger.info(f"Taille des lots : {taille_lot}")
    
    meilleures_metriques = {
        'precision': 0,
        'rappel': 0,
        'f1': 0,
        'epoque': 0
    }

    for epoque in range(nombre_epoques):
        modele.train()
        perte_totale = 0
        predictions_totales = []
        etiquettes_totales = []

        with tqdm(chargeur_donnees_entrainement, 
                 desc=f"Époque {epoque+1}/{nombre_epoques}",
                 unit="lot") as iterateur_epoque:
            
            for etape, lots in enumerate(iterateur_epoque):
                # Passe avant et calcul de la perte
                perte, liste_logits = passe_avant_selection_gloses(modele, lots, appareil)
                
                # Collecte des prédictions
                for logits_lot, lot in zip(liste_logits, lots):
                    predictions = (logits_lot > 0.5).cpu().numpy().astype(int)
                    etiquettes = lot[3].cpu().numpy()
                    predictions_totales.extend(predictions)
                    etiquettes_totales.extend(etiquettes)

                # Rétropropagation et optimisation
                optimiseur.zero_grad()
                perte.backward()
                torch.nn.utils.clip_grad_norm_(modele.parameters(), 1.0)
                optimiseur.step()
                plannificateur.step()

                perte_totale += perte.item()
                
                # Calcul des métriques intermédiaires
                precision_courante = precision_score(
                    etiquettes_totales, 
                    predictions_totales, 
                    zero_division=0
                )
                
                # Mise à jour de la barre de progression
                iterateur_epoque.set_postfix({
                    'Perte': f'{perte.item():.4f}',
                    'Précision': f'{precision_courante:.4f}',
                    'Taux Apprentissage': f'{plannificateur.get_last_lr()[0]:.6f}'
                })

        # Évaluation de fin d'époque
        precision = precision_score(etiquettes_totales, predictions_totales, zero_division=0)
        rappel = recall_score(etiquettes_totales, predictions_totales, zero_division=0)
        f1 = f1_score(etiquettes_totales, predictions_totales, zero_division=0)

        # Mise à jour des meilleures métriques
        if f1 > meilleures_metriques['f1']:
            meilleures_metriques.update({
                'precision': precision,
                'rappel': rappel,
                'f1': f1,
                'epoque': epoque + 1
            })
            
            # Sauvegarde du meilleur modèle
            os.makedirs(repertoire_sortie, exist_ok=True)
            modele.save_pretrained(repertoire_sortie)
            tokeniseur.save_pretrained(repertoire_sortie)

        # Affichage des résultats de l'époque
        logger.info(f"\nRésultats de l'époque {epoque+1}:")
        logger.info(f"Précision: {precision:.4f}")
        logger.info(f"Rappel: {rappel:.4f}")
        logger.info(f"Score F1: {f1:.4f}")

    # Résumé final de l'entraînement
    logger.info("\n========== Entraînement terminé ==========")
    logger.info(f"Meilleures métriques (époque {meilleures_metriques['epoque']}):")
    logger.info(f"Précision: {meilleures_metriques['precision']:.4f}")
    logger.info(f"Rappel: {meilleures_metriques['rappel']:.4f}")
    logger.info(f"Score F1: {meilleures_metriques['f1']:.4f}")
    logger.info(f"Modèle sauvegardé dans : {repertoire_sortie}")

    return meilleures_metriques

In [7]:
def ecrire_predictions(repertoire_sortie, chemin_evaluation, predictions, suffixe=None):
    """
    Écrit les prédictions dans un fichier de sortie CSV.

    Le système sauvegarde les résultats du modèle avec un nommage flexible.
    """
    os.makedirs(repertoire_sortie, exist_ok=True)

    nom_base = os.path.splitext(os.path.basename(chemin_evaluation))[0]
    nom_fichier_sortie = f"{nom_base}_predictions"
    if suffixe:
        nom_fichier_sortie += f"_{suffixe}"
    nom_fichier_sortie += ".csv"

    chemin_complet_sortie = os.path.join(repertoire_sortie, nom_fichier_sortie)

    try:
        with open(chemin_evaluation, 'r', newline='', encoding='utf-8') as fichier_eval:
            lecteur = csv.reader(fichier_eval)
            donnees_originales = list(lecteur)

        for i, prediction in enumerate(predictions):
            if i < len(donnees_originales):
                donnees_originales[i].append(str(prediction))

        with open(chemin_complet_sortie, 'w', newline='', encoding='utf-8') as fichier_sortie:
            ecrivain = csv.writer(fichier_sortie)
            ecrivain.writerows(donnees_originales)

        logger.info(f"Prédictions écrites dans {chemin_complet_sortie}")

    except Exception as e:
        logger.error(f"Erreur lors de l'écriture des prédictions : {e}")

def evaluer_wsd(
    modele,
    tokeniseur,
    chemin_evaluation,
    longueur_sequence_max=128,
    taille_lot_evaluation=16,
    repertoire_sortie= '/kaggle/working/resultEval',
    suffixe=None,
    debut_plage_evaluation=None,
    fin_plage_evaluation=None,
    taille_max_evaluation=None
):
    """
    Évalue un modèle de désambiguïsation lexicale (WSD).

    Le système calcule les performances détaillées sur un jeu de données de test.
    """
    jeu_donnees_evaluation = charger_jeu_donnees(
        chemin_evaluation,
        tokeniseur,
        longueur_sequence_max,
        debut_plage_evaluation=None,
        fin_plage_evaluation=None,
        taille_max_evaluation=None
    )

    echantillonneur = SequentialSampler(jeu_donnees_evaluation)
    chargeur_donnees_evaluation = DataLoader(
        jeu_donnees_evaluation,
        sampler=echantillonneur,
        batch_size=taille_lot_evaluation,
        collate_fn=regrouper_lots
    )

    logger.info("***** Début de l'évaluation *****")
    logger.info(f"Nombre d'exemples : {len(jeu_donnees_evaluation)}")
    logger.info(f"Taille des lots : {taille_lot_evaluation}")

    perte_evaluation = 0.0
    nombre_etapes_evaluation = 0
    predictions = []
    verites_terrain = []
    appareil = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    modele.to(appareil)
    modele.eval()

    for lots in tqdm(chargeur_donnees_evaluation, desc="Évaluation en cours"):
        with torch.no_grad():
            perte, liste_logits = passe_avant_selection_gloses(modele, lots, appareil)

        perte_evaluation += perte
        
        # Collecte des prédictions et des vérités de terrain
        for logits_lot, lot in zip(liste_logits, lots):
            predictions.extend((logits_lot > 0.5).cpu().numpy().astype(int))
            verites_terrain.extend(lot[3].cpu().numpy())
        
        nombre_etapes_evaluation += 1

    perte_evaluation = perte_evaluation / nombre_etapes_evaluation

    # Calcul des métriques détaillées
    precision = precision_score(verites_terrain, predictions, zero_division=0)
    rappel = recall_score(verites_terrain, predictions, zero_division=0)
    f1 = f1_score(verites_terrain, predictions, zero_division=0)
    matrice_confusion = confusion_matrix(verites_terrain, predictions)

    # Écriture des prédictions
    ecrire_predictions(repertoire_sortie, chemin_evaluation, predictions, suffixe='prediction_v1')

    # Journalisation des résultats détaillés
    logger.info("\n📊 Métriques d'évaluation détaillées:")
    logger.info(f"Perte d'évaluation : {perte_evaluation:.4f}")
    logger.info(f"Précision : {precision:.4f}")
    logger.info(f"Rappel : {rappel:.4f}")
    logger.info(f"Score F1 : {f1:.4f}")
    logger.info("\nMatrice de confusion :")
    logger.info(np.array2string(matrice_confusion, separator=', '))

    # Création d'un rapport au format CSV
    rapport_metriques = [
        ["Métrique", "Valeur"],
        ["Perte d'évaluation", f"{perte_evaluation:.4f}"],
        ["Précision", f"{precision:.4f}"],
        ["Rappel", f"{rappel:.4f}"],
        ["Score F1", f"{f1:.4f}"]
    ]

    chemin_rapport = os.path.join(repertoire_sortie, f"rapport_evaluation_{suffixe or 'defaut'}.csv")
    with open(chemin_rapport, 'w', newline='', encoding='utf-8') as fichier:
        ecrivain = csv.writer(fichier)
        ecrivain.writerows(rapport_metriques)
    
    logger.info(f"Rapport d'évaluation sauvegardé dans {chemin_rapport}")

    return {
        'perte': perte_evaluation,
        'precision': precision,
        'rappel': rappel,
        'f1_score': f1,
        'matrice_confusion': matrice_confusion
    }

def obtenir_predictions(modele, tokeniseur, phrase):
    """
    Obtient les prédictions de sens pour un mot ambigu dans une phrase.

    Le système analyse les différents sens possibles et leurs probabilités.
    """
    resultat = re.search(r"\[TGT\](.*)\[TGT\]", phrase)
    if resultat is None:
        print("\nFormat d'entrée incorrect. Veuillez réessayer.")
        return

    mot_ambigu = resultat.group(1).strip()
    cles_sens = []
    definitions = []
    for cle_sens, definition in obtenir_glosses(mot_ambigu, None).items():
        cles_sens.append(cle_sens)
        definitions.append(definition)

    LONGUEUR_SEQUENCE_MAX = 128
    enregistrement = GlossSelectionRecord("test", phrase, cles_sens, definitions, [-1])
    caracteristiques = creer_caracteristiques_depuis_enregistrements([enregistrement], LONGUEUR_SEQUENCE_MAX, tokeniseur)[0]

    appareil = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    modele.to(appareil)

    with torch.no_grad():
        logits = torch.zeros(len(definitions), dtype=torch.double).to(appareil)
        for i, entree_bert in tqdm(list(enumerate(caracteristiques)), desc="Progression"):
            logits[i] = modele.couche_classement(
                modele.bert(
                    input_ids=torch.tensor(entree_bert.ids_entree, dtype=torch.long).unsqueeze(0).to(appareil),
                    attention_mask=torch.tensor(entree_bert.masque_entree, dtype=torch.long).unsqueeze(0).to(appareil),
                    token_type_ids=torch.tensor(entree_bert.ids_segment, dtype=torch.long).unsqueeze(0).to(appareil)
                )[1]
            )
        scores = softmax(logits, dim=0)

    return sorted(zip(cles_sens, definitions, scores), key=lambda x: x[-1], reverse=True)

In [8]:
def fonction_principale():
    """
    Fonction principale pour charger le modèle et lancer l'interaction utilisateur.

    Le système permet de tester les prédictions de désambiguïsation lexicale.
    """
    print("Chargement du modèle...")
    modele = BertWSD.from_pretrained("/kaggle/input/modell")
    tokeniseur = BertTokenizer.from_pretrained("/kaggle/input/modell")
    appareil = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    modele.to(appareil)
    modele.eval()

    while True:
        phrase = input("\nEntrez une phrase avec un mot ambigu entre balises [TGT]\n> ")
        predictions = obtenir_predictions(modele, tokeniseur, phrase)
        if predictions:
            print("\nPrédictions:")
            print(tabulate(
                [[f"{i+1}.", cle, gloss, f"{score:.5f}"] for i, (cle, gloss, score) in enumerate(predictions)],
                headers=["N°", "Clé de sens", "Définition", "Score"])
            )

In [ ]:
if __name__ == '__main__':
    # Options d'exécution
    #choix = input("Que voulez-vous faire ?\n1. Tester des prédictions interactivement\n2. Entraîner le modèle\n3. Évaluer le modèle\nVotre choix : ")

    if choix == '1':
        fonction_principale()
    elif choix == '2':
        resultats = entrainer_wsd(
        chemin_entrainement= '/kaggle/input/dataset/corpus_dir-max_num_gloss5-augmented.csv',
        chemin_evaluation= '/kaggle/input/dataset/semeval2007-max_num_gloss5-augmented.csv',
        debut_plage_entrainement=50001,
        fin_plage_entrainement= 263631,
        taille_max_entrainement=150000,
        debut_plage_evaluation=2001,
        fin_plage_evaluation=38050,
        taille_max_evaluation=22000
        )
    
        print(f"Meilleur score F1 obtenu : {resultats['f1']:.4f}")
        
    elif choix == '3':
        modele = BertWSD.from_pretrained('/kaggle/working/resultats')
        tokeniseur = BertTokenizer.from_pretrained('/kaggle/working/resultats')
        resultats = evaluer_wsd(
            modele=modele,
            tokeniseur=tokeniseur,
            chemin_evaluation='/kaggle/input/dataset/semeval2007-max_num_gloss5-augmented.csv',
            debut_plage_evaluation=None,
            fin_plage_evaluation=None,
            taille_max_evaluation=None
    )

        # Affichage des métriques si nécessaire
        print("Précision:", resultats['precision'])
        print("Rappel:", resultats['rappel'])
        print("Score F1:", resultats['f1_score'])
        print("Matrice de Confusion:\n", resultats['matrice_confusion'])
    else:
        print("Choix invalide.")

Que voulez-vous faire ?
1. Tester des prédictions interactivement
2. Entraîner le modèle
3. Évaluer le modèle
Votre choix :  2


Some weights of the model checkpoint at /kaggle/input/modell were not used when initializing BertWSD: ['ranking_linear.bias', 'ranking_linear.weight']
- This IS expected if you are initializing BertWSD from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWSD from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertWSD were not initialized from the model checkpoint at /kaggle/input/modell and are newly initialized: ['couche_classement.bias', 'couche_classement.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Conversion des données: 100%|██████████| 150000/150000 [02:51<00:00, 877.08it/s] 
/opt/conda/lib/python3.10/site

In [ ]:
sss